## BOID GNN Experimentation

In [ ]:
VISUALIZE = False
NUM_EXPERIMENTS = 100
N_BOIDS=50

In [ ]:
from simulation.Boids import *
import simulation.Boids
import pygame

if VISUALIZE:
    pygame.init()
    clock = pygame.time.Clock()

simScreen = None

for i in range(NUM_EXPERIMENTS):
    if VISUALIZE:
        simScreen = pygame.display.set_mode((WIDTH, HEIGHT))
        simScreen.fill((255, 255, 255))
    simflock = Flock(N_BOIDS, simScreen, VISUALIZE)

    step = 0
    fileData = ""
    while True:
        if VISUALIZE:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
        simS = simflock.run(step)
        fileData += simS

        step += 1
        if VISUALIZE:
            pygame.display.update()
            clock.tick(FPS)
        if step >= (SECONDS * FPS):
            print("Done with " + str(i+1) + " out of " + str(NUM_EXPERIMENTS))
            break

    with open("./data/boid-logs/experiment-" + str(i) + "-log.csv", "w") as f:
        f.write(fileData)

In [ ]:
###############################
####       Pre-reqs        ####
###############################

import numpy as np
import torch
from torch_geometric.data import Data, DataLoader
from torch.autograd import Variable
from matplotlib import pyplot as plt
from os import listdir, remove
from os.path import join, isfile
import pickle
import shutil
import torch_geometric
from utils.BoidDataset import BoidDataset
from tqdm import tqdm
from time import sleep
%matplotlib inline
from random import random

USE_NORMAL_DIST = True

print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###############################
###  Gather and split data  ###
###############################

ta = 70 # Percentage used for training
te = 15 # Percentage used for testing (remaining used for validation)

try:
    shutil.rmtree("/tmp/BOIDs", ignore_errors=False, onerror=None)
except FileNotFoundError as e:
    pass
    
x_dataset = BoidDataset(getXData=True,  root='/tmp/BOIDs').shuffle()
y_dataset = BoidDataset(getXData=False, root='/tmp/BOIDs').shuffle()

In [ ]:
print(len(x_dataset))
print(x_dataset)

In [ ]:
import math
plt.rcParams['figure.figsize'] = [12, 8]

numPoints = 10000

ranData = np.random.permutation(len(x_dataset)-1)

cutDownXData = []
cutDownYData = []
for i in range(numPoints):
    cutDownXData.append(x_dataset[int(ranData[i])])
for i in range(numPoints):
    cutDownYData.append(y_dataset[int(ranData[i])])


In [ ]:
import numpy as np
import torch
from torch import nn
from torch.functional import F
from torch.optim import Adam
from torch_geometric.nn import MetaLayer, MessagePassing, GCNConv
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Softplus, Tanh
from torch.autograd import Variable, grad

import numpy as onp

from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

VERBOSE = False

onp.random.seed(200)
recorded_models = []

plt.rcParams['figure.figsize'] = [8, 5]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class EdgeGNN(MessagePassing):
    def __init__(self, data, hidden=300, aggr='add'):
        
        inLen = len(data.x[0])
        outLen = len(data.y[0])
        edgeLen = len(data.edge_attr[0])
        msg_dim = 100
        
        super(EdgeGNN, self).__init__(aggr=aggr)
        
        self.msg_fnc = Seq(
            Lin(2*inLen+edgeLen, hidden),
            ReLU(),
            Lin(hidden, hidden),
            ReLU(),
            Lin(hidden, hidden),
            ReLU(),
            Lin(hidden, msg_dim)
        )
        
        self.node_fnc = Seq(
            Lin(msg_dim+inLen, hidden),
            ReLU(),
            Lin(hidden, hidden),
            ReLU(),
            Lin(hidden, hidden),
            ReLU(),
            Lin(hidden, outLen)
        )
        
    
    def forward(self, x, edge_index, edge_attr):
        edge_index = torch.tensor([edge_index[1].tolist(), edge_index[0].tolist()], dtype=torch.long).to(device)
        if VERBOSE:
            print("(flipping edge indexs)")
            print("---------------------------")
            print("Inside FORWARD")
            print("=== Node Embeddings ===")
            for i in range(len(x)):
                print("node" + str(i) + ":: " + str(x[i].item()))
            print("=== Edge Embeddings ===")
            for i in range(len(edge_attr)):
                print("From node" + str(edge_index[0][i].item()) + " to node" + str(edge_index[1][i].item()) + ":: " + str(edge_attr[i]))
        result = self.propagate(edge_index, x=x, edge_attr=edge_attr)
        if VERBOSE:
            print("=== Results ===")
            print(len(result))
            print(len(result[0]))
            print(result)
        return result
            
    def message(self, x_i, x_j, edge_attr):
        tmp = torch.cat([x_i, x_j, edge_attr], dim=1)
        if VERBOSE:
            print("---------------------------")
            print("Inside MESSAGE")
            print(x_i)
            print(x_j)
            print(edge_attr)
            print(tmp)
        return self.msg_fnc(tmp)
    
    def update(self, aggr_out, x=None):
        tmp = torch.cat([x, aggr_out], dim=1)
        if VERBOSE:
            print("---------------------------")
            print("Inside UPDATE")
            print(aggr_out)
            print(x)
            print(tmp)
        return self.node_fnc(tmp)
    
    def loss(self, actual, pred):
        retVal = torch.sum(torch.abs(actual[0][0]-pred[0][0]))
        if VERBOSE:
            print("---------------------------")
            print("Inside LOSS")
            print("Actual")
            print(actual[0][0])
            print("Pred")
            print(pred[0][0])
            print("Return")
            print(retVal)
        return retVal

    
    
ytrainLoader = DataLoader(y_dataset, batch_size=32, shuffle=True)
xtrainLoader = DataLoader(x_dataset, batch_size=32, shuffle=True)

xHist = []
yHist = []
for data in xtrainLoader:
    xHist.append(data.y[0][0].item())
for data in ytrainLoader:
    yHist.append(data.y[0][0].item())

LR = 0.0001

xMod = EdgeGNN(cutDownXData[0])
yMod = EdgeGNN(cutDownYData[0])

xMod = xMod.to(device)
yMod = yMod.to(device)

xMod.train()
yMod.train()

xOptimizer = torch.optim.Adam(xMod.parameters(), lr=LR, weight_decay=5e-8)
yOptimizer = torch.optim.Adam(yMod.parameters(), lr=LR, weight_decay=5e-8)

num_epochs = 30
epoch = 0
losses = []

for epoch in tqdm(range(epoch, num_epochs)):

    xMod.to(device)
    yMod.to(device)

    xMod.train()
    yMod.train()

    x_total_loss = 0.0
    y_total_loss = 0.0
    
    xPredHist = []
    yPredHist = []
    
    num_items = 0
    i=0
    total_loss = 0
    
    for data in xtrainLoader:

        xOptimizer.zero_grad()
        
        data.x = data.x.to(device)
        data.edge_index = data.edge_index.to(device)
        data.edge_attr = data.edge_attr.to(device)
        data.y = data.y.to(device)
        data.batch = data.batch.to(device)
        
        xPred = xMod(data.x, data.edge_index, data.edge_attr)
        xPredHist.append(xPred[0][0].item())
        xLoss = xMod.loss(data.y, xPred)
        (xLoss/int(data.batch[-1]+1)).backward()
        x_total_loss += xLoss.item()
        xOptimizer.step()
    
        total_loss += xLoss.item()
        i += 1
        num_items += int(data.batch[-1]+1)

    for data in ytrainLoader:
    
        yOptimizer.zero_grad()
        yPred = yMod(data.x.to(device), data.edge_index.to(device), data.edge_attr.to(device))
        yPredHist.append(yPred[0][0].item())
        yLoss = yMod.loss(data.y.to(device), yPred)
        (yLoss/int(data.batch.to(device)[-1]+1)).backward()
        y_total_loss += yLoss.item()
        yOptimizer.step()

        if VERBOSE:
            raise Exception


    cur_loss = total_loss/num_items
    print("{during: " + str(cur_loss) + "}")
    
    fig, (ax1, ax2) = plt.subplots(1, 2)

    ax1.hist(xHist, 500, density=True, facecolor='g', alpha=0.25)
    ax1.hist(xPredHist, 500, density=True, facecolor='b', alpha=0.25)
    
    ax2.hist(yHist, 500, density=True, facecolor='g', alpha=0.25)
    ax2.hist(yPredHist, 500, density=True, facecolor='b', alpha=0.25)
    
    fig.show()
    plt.show()
    
    losses.append([cur_loss])#, y_total_loss])
    sleep(0.5)
    
    torch.save(xMod.state_dict(), './models/boid-models/xBOID'+str(epoch)+'.mod')
    torch.save(yMod.state_dict(), './models/boid-models/yBOID'+str(epoch)+'.mod')
    
torch.save(xMod.state_dict(), './models/boid-models/xBOID_FINAL.mod')
torch.save(yMod.state_dict(), './models/boid-models/yBOID_FINAL.mod')

In [ ]:
plt.plot([i for i in range(len(losses))],[losses[i][0] for i in range(len(losses))])

In [ ]:
nNeighbors = 1
numPoints = 5000

from random import random
import math
from math import sqrt

dataListX = []
dataListY = []
dataListXA = []
dataListYA = []

visRange = 100
MAXSPEED = 3
results = []

for i in range(numPoints):
    rowX = []
    rowY = []
    
    data = [[1.] for _ in range(nNeighbors+1)]
    
    myX = random() * 400
    myY = random() * 400
    myT = random()*2*math.pi
    myS = random() * MAXSPEED
    myXa = math.cos(myT) * myS
    myYa = math.sin(myT) * myS
        
    e1 = []
    e2 = []
    att = []
    for j in range(1, nNeighbors+1):  
        
        distance = random() * visRange
        speed = random() * MAXSPEED
        azi = random()*2*math.pi
        theta = random()*2*math.pi
        inXa = math.cos(theta)*speed
        inYa = math.sin(theta)*speed
        inX = (myX + distance*math.cos(azi))
        inY = (myY + distance*math.sin(azi))
        
        x12 =(inX-myX)*(myXa/myS) - (inY-myY)*(myYa/myS)
        y12 =(inX-myX)*(myYa/myS) + (inY-myY)*(myXa/myS)
        xd12 = (inXa-myXa)
        yd12 = (inYa-myYa)
        d = distance
        
        # Conversion to standard naming
        lFX = x12
        lFY = y12
        lFXD = xd12
        lFYD = yd12
        d = sqrt(pow(lFX, 2) + pow(lFY, 2))
        s = sqrt(pow(lFXD, 2) + pow(lFYD, 2))
        invd = 1.0 / d
        invs = 0
        if s != 0:
            invs = 1.0 / s
        lFnX = lFX * invd
        lFnY = lFY * invd
        lFnXD = lFXD * invs
        lFnYD = lFYD * invs
        
        e1.extend([j])
        e2.extend([0])
        att.append([lFX, lFY, d, invd, lFnX, lFnY, lFXD, lFYD, s, invs, lFnXD, lFnYD])
        rowX.extend([lFX, d, invd, lFnX, lFXD, s, invs, lFnXD])
        rowY.extend([lFY, d, invd, lFnY, lFYD, s, invs, lFnYD])
    
    x = torch.tensor(data, dtype=torch.float).to(device)
    e = torch.tensor([e1, e2], dtype=torch.long).to(device)
    a = torch.tensor(att, dtype=torch.float).to(device)
    
    xPred = xMod(x, e, a)
    yPred = yMod(x, e, a)
    newXVel = (xPred[0][0].item() - 0.5) * 4.0
    newYVel = (yPred[0][0].item() - 0.5) * 8.0
    
    xaData = []
    xaData.extend(rowX)
    xaData.append(newXVel)
    
    yaData = []
    yaData.extend(rowY)
    yaData.append(newYVel)
        
    dataListXA.append(xaData)
    dataListYA.append(yaData)
    dataListYA.append(xaData)

xaFile = ''
for i in range(len(dataListXA)): 
    xaStr = ''
    for j in range(len(dataListXA[i])):
        xaStr += str(dataListXA[i][j]) + ","
    xaStr = xaStr[:-1] + "\n"
    xaFile+=xaStr
    
yaFile = ''
for i in range(len(dataListYA)): 
    yaStr = ''
    for j in range(len(dataListYA[i])):
        yaStr += str(dataListYA[i][j]) + ","
    yaStr = yaStr[:-1] + "\n"
    yaFile+=yaStr
    
with open("./data/MME-data/xBOIDoutdata.csv", 'w') as f:
    f.write(xaFile)
    print("writing")
with open("./data/MME-data/yBOIDoutdata.csv", 'w') as f:
    f.write(yaFile)
    print("writing")

print("DONE")